### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_metadata, males_keys, females_keys

H5_dic = read_metadata()
chunk_size = 512*10

males = H5PY_RW('test_raw_16k.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW('test_raw_16k.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
N = 256
max_pool = 128
batch_size = 8
learning_rate = 0.001

config_model = {}
config_model["type"] = "DPCL_train_front"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.01

idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_finetuning"
config_model["batch_size"] = 1

In [5]:
from models.adapt import Adapt
import config
full_id = 'wispy-forest-8617'+idd

folder='DPCL_fine_tuning'
model = Adapt(config_model=config_model,pretraining= False, folder=folder)
path = os.path.join(config.workdir, 'log', 'DPCL_train_front')

from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.sepNet.output = model.sepNet.prediction
    model.create_saver()
    model.restore_model(path, full_id)
    model.sepNet.output = model.sepNet.separate
    model.separator
    model.back
    model.cost
    model.optimize
    model.tensorboard_init()



ID : AdaptiveNet-crimson-frog-5697
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/DPCL_train_front/AdaptiveNet-wispy-forest-8617-N=256--alpha=0.001--batch_size=8--beta=0.1--chunk_size=5120--maxpool=128--reg=0.0001--rho=0.01--smooth_size=20--type=DPCL_train_front--window=1024-/model.ckpt
Tensor("separate/Reshape_5:0", shape=(?, ?, 256, 1), dtype=float32)
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 20, 256, 256) dtype=float32_ref>, <tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable '

## Connect DPCL model to the front end and back end

In [6]:
from itertools import compress
with model.graph.as_default():
    global_vars = tf.global_variables()
    is_not_initialized = model.sess.run([~(tf.is_variable_initialized(var)) \
                                   for var in global_vars])
    not_initialized_vars = list(compress(global_vars, is_not_initialized))
    print not_initialized_vars
    if len(not_initialized_vars):
        init = tf.variables_initializer(not_initialized_vars)


[<tf.Variable 'optimize/conv/W/RMSProp:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'optimize/conv/W/RMSProp_1:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'optimize/smooth/smoothing_filter/RMSProp:0' shape=(1, 20, 256, 256) dtype=float32_ref>, <tf.Variable 'optimize/smooth/smoothing_filter/RMSProp_1:0' shape=(1, 20, 256, 256) dtype=float32_ref>]


# Model creation

# Pretraining the model 

In [7]:
nb_iterations = 1000
#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 1.1536
DAS model saved at iteration number  0  with cost =  1.1536
Step # 1  loss= 0.927743
Step # 2  loss= 0.931067
Step # 3  loss= 0.227723
Step # 4  loss= 0.972699
Step # 5  loss= 1.09358
Step # 6  loss= 0.594789
Step # 7  loss= 1.2961
Step # 8  loss= 1.20904
Step # 9  loss= 0.615677
Step # 10  loss= 0.719588
Step # 11  loss= 0.660654
Step # 12  loss= 0.915523
Step # 13  loss= 0.61194
Step # 14  loss= 0.893956
Step # 15  loss= 1.06256
Step # 16  loss= 0.578016
Step # 17  loss= 0.622088
Step # 18  loss= 0.698267
Step # 19  loss= 0.731227
Step # 20  loss= 0.855751
DAS model saved at iteration number  20  with cost =  0.855751
Step # 21  loss= 0.566559
Step # 22  loss= 0.781461
Step # 23  loss= 0.678661
Step # 24  loss= 0.805829
Step # 25  loss= 0.822999
Step # 26  loss= 1.16017
Step # 27  loss= 0.721024
Step # 28  loss= 0.831858
Step # 29  loss= 0.623625
Step # 30  loss= 1.07433
Step # 31  loss= 0.584814
Step # 32  loss= 0.80182
Step # 33  loss= 0.589364
Step # 34  los

InvalidArgumentError: Nan in summary histogram for: conv_1/summaries/histogram
	 [[Node: conv_1/summaries/histogram = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](conv_1/summaries/histogram/tag, conv/W/read/_139)]]

Caused by op u'conv_1/summaries/histogram', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self)